In [16]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/brandonmanley/Desktop/PhD/oam_pheno/dijet_dsa')
import dsa_mc.dijet as dijet
import random
from IPython.display import display, Math
from tabulate import tabulate

# setup plotting
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 16
plt.rcParams["legend.fontsize"] = 12
plt.rcParams["axes.labelsize"] = 16  
plt.rcParams["xtick.labelsize"] = 14  
plt.rcParams["ytick.labelsize"] = 14
plt.rcParams["axes.titlesize"] = 18  
plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{amsmath}"
%config InlineBackend.figure_format = 'retina'

In [17]:
dj = dijet.DIJET(fit_type='ones', lambdaIR=0.3)


--> loaded unpol. amp. data from /dipoles/narr_ymin4.61_ymax14.91_AAMS09.dat
--> loaded pol. amp. data from /dipoles/d05-rc/
--> loaded params from /dipoles/replica_params_ones.csv
--> loaded random moment params from /dipoles/moment_params_ones.csv
--> loaded replica 1


In [18]:
# matching fourier-bessel 
kins = dijet.Kinematics(x=0.01, z=0.01, Q=10, pT=10)
print(dj.fourier_bessel(kins, [[1,1,1,1]], 'N'))
print(dj.fourier_bessel(kins, [[1,1,1,1]], 'Qu'))

-3.1843025584539455
318.8384682301659


In [19]:
# matching coefficients
kins = dijet.Kinematics(x=0.01, z=0.3, Q=10, pT=2, y=0.1, delta=0.5)
kins.s = (kins.Q**2)/(kins.x*kins.y)

coeffs = ['A_TT', 'B_TT', 'A_LT', 'B_LT', 'C_LT']
coeffs += ['A_TT_unpolar', 'B_TT_unpolar', 'A_TmT_unpolar', 'B_TmT_unpolar', 'C_TmT_unpolar', 'A_LL_unpolar', 'B_LL_unpolar']
coeff_values = {coeff: dj.get_coeff(coeff, kins) for coeff in coeffs}

ben_coeffs = {
    'A_TT': -8.494285245553912e-07,
    'B_TT': -3.444049066922065e-08,
    'A_LT': 2.356197548067663e-05,
    'B_LT': 8.214268057773021e-05,
    'C_LT': -9.16784882331537e-07,
    'A_TT_unpolar': 1.4975559569144442e-06,
    'B_TT_unpolar': -1.8843366541503538e-07,
    'A_TmT_unpolar': -1.0844370722483909e-06,
    'B_TmT_unpolar': 1.3645196461088766e-07,
    'C_TmT_unpolar': -4.3377482889935633e-07,
    'A_LL_unpolar': 1.615215249961842e-06,
    'B_LL_unpolar':6.061657486205812e-07
}

for coeff in coeffs:
    z = ((ben_coeffs[coeff] - coeff_values[coeff])/coeff_values[coeff])*100 
    print(coeff, z)

A_TT -0.026270469113025767
B_TT -96.50579016205518
A_LT 22897.688201083827
B_LT 51371.643465722766
C_LT 11.83284395370989
A_TT_unpolar -0.02627046915190597
B_TT_unpolar -76.0768663289469
A_TmT_unpolar -0.02627046915186424
B_TmT_unpolar -191.7629516089064
C_TmT_unpolar -0.02627046915187644
A_LL_unpolar -0.02627046915520491
B_LL_unpolar -0.026270469152466197


In [20]:
# matching final results
kins = dijet.Kinematics(x=0.01, z=0.3, Q=10, pT=2, y=0.1, delta=0.5)
kins.s = (kins.Q**2)/(kins.x*kins.y)

weights = ['1', 'cos(phi_kp)', 'cos(phi_Dp)', 'cos(phi_Dp)cos(phi_kp)', 'sin(phi_Dp)sin(phi_kp)']
nums = [dj.get_angle_integrated_xsec(kins, weight=w, kind='num') for w in weights]
den = dj.get_angle_integrated_xsec(kins, weight='1', kind='den')
corrs = {w: nums[iw]/den for iw, w in enumerate(weights)}


ben_corrs = {
    '1': -0.018930764988144055,
    'cos(phi_kp)': 0.18539827064591968,
    'cos(phi_Dp)': -9.594462866617396e-05,
    'cos(phi_Dp)cos(phi_kp)': 0.08079283790222129,
    'sin(phi_Dp)sin(phi_kp)': -0.0009017194455850274
}


for w in weights:
    z = ((ben_corrs[w] - corrs[w])/corrs[w])*100 
    print(w, round(ben_corrs[w],8) , round(corrs[w], 8), round(z, 4))

1 -0.01893076 -0.01893076 0.0
cos(phi_kp) 0.18539827 0.00080595 22903.7314
cos(phi_Dp) -9.594e-05 -0.0027451 -96.5049
cos(phi_Dp)cos(phi_kp) 0.08079284 0.00015692 51385.1689
sin(phi_Dp)sin(phi_kp) -0.00090172 -0.0008061 11.8622


In [23]:
coeff_eqs = ['13a','13b','15a','15b','15c','23a','23b', '23c','23d','23e','23f','23g']
corr_eqs = ['25a','25b','25c','25d','25e']

table_data = [[eq, bv, mv, bv/mv] for eq, bv, mv in zip(coeff_eqs, ben_coeffs.values(), coeff_values.values())]
table_data += [[eq, bv, mv, bv/mv] for eq, bv, mv in zip(corr_eqs, ben_corrs.values(), corrs.values())]

headers = ['eqn.', 'ben', 'me', 'ben/me']

# print(tabulate(table_data, headers=headers, tablefmt="grid"))
print(tabulate(table_data, headers=headers))




eqn.             ben            me       ben/me
------  ------------  ------------  -----------
13a     -8.49429e-07  -8.49652e-07    0.999737
13b     -3.44405e-08  -9.85645e-07    0.0349421
15a      2.3562e-05    1.02454e-07  229.977
15b      8.21427e-05   1.59588e-07  514.716
15c     -9.16785e-07  -8.19781e-07    1.11833
23a      1.49756e-06   1.49795e-06    0.999737
23b     -1.88434e-07  -7.87663e-07    0.239231
23c     -1.08444e-06  -1.08472e-06    0.999737
23d      1.36452e-07  -1.487e-07     -0.91763
23e     -4.33775e-07  -4.33889e-07    0.999737
23f      1.61522e-06   1.61564e-06    0.999737
23g      6.06166e-07   6.06325e-07    0.999737
25a     -0.0189308    -0.0189308      1
25b      0.185398      0.000805949  230.037
25c     -9.59446e-05  -0.0027451      0.0349513
25d      0.0807928     0.000156924  514.852
25e     -0.000901719  -0.000806098    1.11862
